In [2]:
!pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.5 MB 11.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.5 MB 11.7 MB/s eta 0:00:01
   ----------------------- ---------------- 6.8/11.5 MB 12.0 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.5 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------- ------------------------------ 2.9/12.6 MB 14.0 MB/s eta 0:00:01
   ------------------ --------------------- 5.8/12.6 MB 14.1 MB/s eta 0:00:01
   ---------------------------- ----------- 8.9/12.6 MB 14.6 MB/s eta 0:00:01
   ------------------------------------- -- 11.8/12.6 MB 14.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 13.9 MB/s eta 0:00:00


In [1]:
import os
import json
from tqdm import tqdm
import pandas as pd
import sqlite3
import numpy as np

In [4]:
json_filename = 'example.json'

count = 0
with open('reddit/submissions/RS_2025-01/RS_2025-01', 'r', encoding='utf-8') as f:
    print(len(f.readlines))
    for line in f:
        line = json.loads(line)
        if len(line['all_awardings']) == 0:
            count += 1
            if count % 10000 == 0:
                print(f'At line {count}')
                print(line['all_awardings'])
            continue
        if not os.path.isfile(json_filename):
            data = []
            with open(json_file, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=4)
            
        with open(json_filename, 'w', encoding='utf-8') as json_file:
            json.dump(line, json_file, indent=4)
        print(f'Number of attributes: {len(line.keys())}')
        break

print(f'Wrote example line to {json_filename}')

PermissionError: [Errno 13] Permission denied: 'reddit/submissions/RS_2025-01/RS_2025-01'

In [3]:
def write_lines(input_file, output_file, lines):
    file_content = []
    
    with open(input_file, 'r', encoding='utf-8') as f:
        for line_count, line in enumerate(tqdm(f, total=lines, desc="Reading lines"), start=1):
            if line_count > lines:
                break
            
            try:
                cleaned_line = json.loads(line)  # Parse JSON
                file_content.append(cleaned_line)
            except:
                continue  # Skip invalid JSON lines
            
    print(f'Writing {output_file}...')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(file_content, f, indent=4, ensure_ascii=False)  # Remove extra backslashes

# Comments

In [3]:
comment_file = 'reddit/comments/RC_2025-01/RC_2025-01'
write_lines(comment_file, 'lines.json', 1000000)

Reading lines: 100%|███████████████| 1000000/1000000 [00:23<00:00, 42263.15it/s]


In [4]:
def unique_authors(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    authors = set()
    for i in data:
        author = i['author']
        if author != '[deleted]':
            authors.add(author)

    return authors

In [13]:
print(len(unique_authors('lines.json')))

460645


In [23]:
def get_deleted_count(json_file):
    with open('lines.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    del_count = 0
    for i in tqdm(data, desc='Reading data...'):
        if i['author'] == '[deleted]':
            del_count += 1
    return del_count

In [25]:
get_deleted_count('lines.json')

Reading data...: 100%|███████████| 1000000/1000000 [00:00<00:00, 1629128.60it/s]


32769

In [8]:
with open('lines.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [8]:
keys = []
for i in data[0].keys():
    print(f'{i}: {data[0][i]}')
    keys.append(i)

_meta: {'retrieved_2nd_on': 1735819210}
all_awardings: []
approved_at_utc: None
approved_by: None
archived: False
associated_award: None
author: Alone_Friendship4618
author_flair_background_color: None
author_flair_css_class: None
author_flair_richtext: []
author_flair_template_id: None
author_flair_text: None
author_flair_text_color: None
author_flair_type: text
author_fullname: t2_94fk7riv
author_is_blocked: False
author_patreon_flair: False
author_premium: False
awarders: []
banned_at_utc: None
banned_by: None
body: Creation change and miracle
can_gild: False
can_mod_post: False
collapsed: False
collapsed_because_crowd_control: None
collapsed_reason: None
collapsed_reason_code: None
comment_type: None
controversiality: 0
created: 1735689600
created_utc: 1735689600
distinguished: None
downs: 0
edited: False
gilded: 0
gildings: {}
id: m4rvtu0
is_submitter: False
likes: None
link_id: t3_1hqah4t
locked: False
mod_note: None
mod_reason_by: None
mod_reason_title: None
mod_reports: []
name

In [4]:
print(pd.__version__)

2.1.4


In [28]:
sqlite_path = 'reddit_db.db'

unique_authors = set()  # Set to keep track of authors
filtered_data = []

for item in tqdm(data, desc='Converting authors'):
    # Check if the 'author' key exists in the item
    if 'author_fullname' in item:
        author = item['author_fullname']
        if author not in unique_authors and item['author'] != '[deleted]':
            unique_authors.add(author)
            filtered_item = {}
            for k, v in item.items():
                if not k.startswith('author'):
                    continue
                if type(v) == dict:
                    v = str(v)
                if type(v) == list:
                    if len(v) == 0:
                        v = None
                    else:
                        v = str(v)
                filtered_item[k] = v 
            # filtered_item = {k: v for k, v in item.items() if k.startswith('author')}
            filtered_data.append(filtered_item)

conn = sqlite3.connect(sqlite_path)
df = pd.DataFrame([{k: list(v) if isinstance(v, np.ndarray) else v for k, v in row.items()} for row in filtered_data])
df.to_sql('authors', conn, if_exists='replace', index=False)
conn.close()
print(f'DB saved to {sqlite_path} successfully!')

Converting authors: 100%|█████████| 1000000/1000000 [00:03<00:00, 282052.59it/s]


DB saved to reddit_db.db successfully!


# Submissions

In [7]:
def write_batch_lines(input_file, output_folder):
 
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Read and write in chunks of 10,000 lines
    batch_size = 10000
    batch_number = 1
    lines_batch = []
    
    with open(input_file, "r", encoding="utf-8") as infile:
        for line in tqdm(infile, desc='Writing lines...'):
            lines_batch.append(line.strip())
    
            # When batch reaches 10,000 lines, write to a JSON file
            if len(lines_batch) == batch_size:
                output_file = os.path.join(output_folder, f"batch_{batch_number}.json")
                with open(output_file, "w", encoding="utf-8") as outfile:
                    json.dump(lines_batch, outfile, indent=4)
                
                # Reset for next batch
                lines_batch = []
                batch_number += 1
    
    # Write remaining lines if any
    if lines_batch:
        output_file = os.path.join(output_folder, f"batch_{batch_number}.json")
        with open(output_file, "w", encoding="utf-8") as outfile:
            json.dump(lines_batch, outfile, indent=4)

In [80]:
def inplace_authors(line, json_file, authors_processed, batch_size=1000):
    if not os.path.isfile(json_file):
        data = []
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4)
    
    line = json.loads(line)
    
    if line['author'] != '[deleted]':
        if line['author_fullname'] not in authors_processed:
            with open(json_file, encoding='utf-8') as f:
                data = json.load(f)
            
            # Extract author-related keys
            author_data = {}
            for k, v in line.items():
                if k.startswith('author'):
                    if isinstance(v, list):
                        v = None if not v else str(v)
                    if isinstance(v, dict):
                        v = str(v)
                author_data[k] = v
            
            # Append the new author data
            data.append(author_data)
            
            # Write back to the JSON file only every batch_size iterations
            if len(data) % batch_size == 0:
                with open(json_file, 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
            
            # Mark this author as processed
            authors_processed.add(line['author_fullname'])
    
    return authors_processed

In [82]:
submission_file = 'reddit/submissions/RS_2025-01/RS_2025-01'
json_file = 'authors.json'
total = 331981952
# write_lines(submission_file, 'lines_sub.json', 1000000)
authors_processed = set([])
with open(submission_file, 'r', encoding='utf-8') as f:
    for line in tqdm(f, desc='Processing authors', total=total):
        authors_processed = inplace_authors(line, json_file, authors_processed)

Processing authors:   0%|                                                                | 0/331981952 [00:00<?, ?it/s]


JSONDecodeError: Expecting ',' delimiter: line 38299 column 192 (char 1558048)

In [ ]:
with open('lines_sub.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [12]:
print(f'In total {len(data[0].keys())} keys')
for i in data[0].keys():
    print(f'{i}:{data[0][i]}')

In total 106 keys
_meta:{'retrieved_2nd_on': 1735819210}
all_awardings:[]
allow_live_comments:False
approved_at_utc:None
approved_by:None
archived:False
author:NBA_MOD
author_flair_background_color:#edeff1
author_flair_css_class:NBA
author_flair_richtext:[{'e': 'text', 't': ' r/NBA'}]
author_flair_template_id:e5aa3fb6-3feb-11e8-8409-0ef728aaae7a
author_flair_text: r/NBA
author_flair_text_color:dark
author_flair_type:richtext
author_fullname:t2_6vjwa
author_is_blocked:False
author_patreon_flair:False
author_premium:False
awarders:[]
banned_at_utc:None
banned_by:None
can_gild:False
can_mod_post:False
category:None
clicked:False
content_categories:None
contest_mode:False
created:1735689600
created_utc:1735689600
discussion_type:None
distinguished:None
domain:self.nba
downs:0
edited:False
gilded:0
gildings:{}
hidden:False
hide_score:False
id:1hqr92b
is_created_from_ads_ui:False
is_crosspostable:True
is_meta:False
is_original_content:False
is_reddit_media_domain:False
is_robot_indexable:Tru

In [45]:
def make_df(data, columns):
    return_data = []
    for i in data:
        filtered_data = {}
        for k, v in i.items():
            if type(v) == list:
                if len(v) == 0:
                    v = None
                else:
                    v = str(v)
            if type(v) == dict:
                v = str(v)
            
            if k in columns:
                filtered_data[k] = v
        return_data.append(filtered_data)

    df_return = pd.DataFrame(return_data)
    return_data = []
    return df_return

In [ ]:
# Define Reddit post data categories
metadata_keys = ["id", "name", "permalink", "url", "created_utc", "retrieved_on", "_meta.retrieved_2nd_on"]
post_content_keys = ["id", "title", "selftext", "domain", "is_self", "is_video", "media", "secure_media"]
engagement_keys = ["num_comments", "score", "ups", "downs", "upvote_ratio", "view_count"]
author_keys = ["author", "author_fullname", "author_premium", "author_flair_text", "author_flair_css_class", "author_flair_richtext", "author_flair_background_color", "author_flair_text_color", "author_flair_type"]
subreddit_keys = ["subreddit", "subreddit_id", "subreddit_name_prefixed", "subreddit_type", "subreddit_subscribers"]
moderation_keys = ["archived", "locked", "stickied", "removed_by", "removed_by_category", "banned_by", "banned_at_utc", "mod_reports", "num_reports", "report_reasons", "removal_reason", "quarantine", "approved_by", "approved_at_utc"]
awards_flair_keys = ["all_awardings", "awarders", "gilded", "total_awards_received", "top_awarded_type", "gildings", "link_flair_text", "link_flair_richtext", "link_flair_type", "link_flair_background_color", "link_flair_text_color", "link_flair_css_class"]
visibility_keys = ["over_18", "spoiler", "pinned", "contest_mode", "hide_score", "hidden", "no_follow", "can_gild", "can_mod_post", "is_crosspostable", "is_robot_indexable", "is_original_content", "is_reddit_media_domain", "is_created_from_ads_ui", "discussion_type", "suggested_sort", "visited", "saved", "send_replies", "pwls", "wls", "media_only", "content_categories", "treatment_tags"]

cols = [metadata_keys, post_content_keys, engagement_keys, author_keys, subreddit_keys, moderation_keys, moderation_keys, awards_flair_keys, visibility_keys]
cols_names = ['metadata_keys', 'post_content_keys', 'engagement_keys', 'author_keys', 'subreddit_keys', 'moderation_keys', 'moderation_keys',  'awards_flair_keys', 'visibility_keys']
conn = sqlite3.connect('reddit_posts.db')
for i in tqdm(range(len(cols)), desc='Making SQL database'):
    col = cols[i]
    col_name = cols_names[i]
    col.insert(0, 'id')
    col = list(set(col))
    df = make_df(data, col) 
    df.to_sql(col_name, conn, if_exists='replace', index=False)
print('SQL database successfully made. Written to reddit_posts.db')

## Find nested JSON

In [21]:
submission_file = 'reddit/submissions/RS_2025-01/RS_2025-01'

def find_non_empty_nested_keys(obj, parent_key=""):
    """Recursively find non-empty nested JSON objects (length > 0), excluding items in list excluding."""
    nested_keys = []

    excluding = ['_meta', 'author_flair_richtext', 'link_flair_richtext', 'preview', 'crosspost_parent_list']
    
    if isinstance(obj, dict):
        for key, value in obj.items():
            if key in excluding:  # Exclude keys in the excluding list
                continue
            if isinstance(value, dict) and len(value) > 0:  # Non-empty dictionary
                nested_keys.append(parent_key + key if parent_key else key)
                nested_keys.extend(find_non_empty_nested_keys(value, parent_key + key + "."))  # Recurse
            elif isinstance(value, list):  # List containing non-empty dicts
                for i, item in enumerate(value):
                    if isinstance(item, dict) and len(item) > 0:
                        nested_keys.append(parent_key + key if parent_key else key)
                        nested_keys.extend(find_non_empty_nested_keys(item, parent_key + key + f"[{i}]."))  # Recurse

    elif isinstance(obj, list):
        for i, item in enumerate(obj):
            if isinstance(item, dict) and len(item) > 0:
                nested_keys.extend(find_non_empty_nested_keys(item, parent_key + f"[{i}]."))  # Recurse

    return nested_keys

with open(submission_file, 'r', encoding='utf-8') as f:
    for line in tqdm(f, desc='Reading lines...'):
        line = json.loads(line)  # Parse JSON
        nested_keys = find_non_empty_nested_keys(line)  # Find non-empty nested JSON
        
        if nested_keys:  # Only print if nested objects have length > 0
            print("\n" + "=" * 80)
            print(f"🔍 Non-empty Nested JSON found in keys: {nested_keys}\n")
            print(json.dumps(line, indent=4, ensure_ascii=False))  # Pretty-print JSON
            print("=" * 80 + "\n")
            break

Reading lines...: 19it [00:00, 8561.64it/s]


🔍 Non-empty Nested JSON found in keys: ['gallery_data', 'gallery_data.items', 'gallery_data.items', 'gallery_data.items', 'media_metadata', 'media_metadata.066pukz7w9ae1', 'media_metadata.066pukz7w9ae1.p', 'media_metadata.066pukz7w9ae1.p', 'media_metadata.066pukz7w9ae1.p', 'media_metadata.066pukz7w9ae1.p', 'media_metadata.066pukz7w9ae1.p', 'media_metadata.066pukz7w9ae1.p', 'media_metadata.066pukz7w9ae1.s', 'media_metadata.ht8b3l08w9ae1', 'media_metadata.ht8b3l08w9ae1.p', 'media_metadata.ht8b3l08w9ae1.p', 'media_metadata.ht8b3l08w9ae1.p', 'media_metadata.ht8b3l08w9ae1.p', 'media_metadata.ht8b3l08w9ae1.p', 'media_metadata.ht8b3l08w9ae1.p', 'media_metadata.ht8b3l08w9ae1.s', 'media_metadata.ira4rxz7w9ae1', 'media_metadata.ira4rxz7w9ae1.p', 'media_metadata.ira4rxz7w9ae1.p', 'media_metadata.ira4rxz7w9ae1.p', 'media_metadata.ira4rxz7w9ae1.p', 'media_metadata.ira4rxz7w9ae1.p', 'media_metadata.ira4rxz7w9ae1.p', 'media_metadata.ira4rxz7w9ae1.s']

{
    "_meta": {
        "retrieved_2nd_on": 173